In [ ]:
# this is to setup the path so we can import the mindpype library
import os; os.sys.path.append(os.path.dirname(os.path.abspath('.')))

In [ ]:
# import mindpype
import mindpype as mp

We will start by importing the training and testing files.

In [ ]:
# get the training and testing files
from glob import glob
files = glob("P:/general_prism/Side Projects/NIRS BCI/Data/Dec4/sub-P003/sourcedata/*.xdf")
print(files)

# training files
training_files = files[1:-1]

# testing files (one from each task)
testing_files = [files[0], files[-1]]

The first step to creating a pipeline is to create a session, which serves as a sandbox for all components in the pipeline.

In [ ]:
# create the mindpype session
s = mp.Session.create()

Next, we will define some session parameters that will be used to create our data sources and nodes.

In [ ]:
# define some session parameters
Fs = 50
Nc = 46
trial_len = 15
Ns = int(trial_len * Fs)

epoch_len = int(Fs * 4)
epoch_stride = int(Fs * 0.1)

Then, we will create our training and test data sources. For our training and test data sources, we will created an epoched XDF file input source using the ```create_epoched()``` factory method. 

For the training data, we will then seperate the input source data into a tensor (containing stream data) and a label tensor (containing marker data) by using the ```load_into_tensors()``` method.

For our test data, we will create a volatile tensor to ingest data from the epoched XDF file input source using the ```create_from_handle()``` factory method.

In [ ]:
# create the data sources

# training data
tr_data_src = mp.source.InputXDFFile.create_epoched(s, training_files, channels=range(Nc),
                                                    tasks=['{"status": "Neutral"}', '{"status": "Music"}'],
                                                    stype='NIRS', Ns=Ns)
t_tr_data, t_tr_labels = tr_data_src.load_into_tensors()

# test data
te_data_src = mp.source.InputXDFFile.create_epoched(s, testing_files, channels=range(Nc),
                                                    tasks=['{"status": "Neutral"}', '{"status": "Music"}'],
                                                    stype='NIRS', Ns=Ns)

# create a volatile data edge to ingest data from the source
t_data_in = mp.Tensor.create_from_handle(s, (Nc, Ns), te_data_src)

We will also create a tensor to to store the prediction that is outputed by the classifier in our pipeline.

In [ ]:
# create the edge that will store the classifier output
t_pred = mp.Tensor.create(s, (1,))

Next, we will create virtual tensors to store all of our intermediate data calculated in our pipeline by using the ```create_virtual()``` factory method. Since these edges represent intermediate data that is only required in the process of completing a calculation and we don't need to access them at a leter point we use the virtual type. 

In [ ]:
# create our virtual edges

v_tensors = [
                mp.Tensor.create_virtual(s),  #  0 - filtered data
                mp.Tensor.create_virtual(s),  #  1 - epoched data
                mp.Tensor.create_virtual(s),  #  2 - mean
                mp.Tensor.create_virtual(s),  #  3 - variance
                mp.Tensor.create_virtual(s),  #  4 - kurtosis
                mp.Tensor.create_virtual(s),  #  5 - skew
                mp.Tensor.create_virtual(s),  #  6 - slope
                mp.Tensor.create_virtual(s),  #  7 - mean+var
                mp.Tensor.create_virtual(s),  #  8 - mean+var+kurt
                mp.Tensor.create_virtual(s),  #  9 - mean+var+kurt+skew
                mp.Tensor.create_virtual(s),  # 10 - mean+var+kurt+skew+slope
                mp.Tensor.create_virtual(s),  # 11 - flattened feature vector
                mp.Tensor.create_virtual(s),  # 12 - normalized feature vector
                mp.Tensor.create_virtual(s),  # 13 - selected features
    ]

Next, we will create objects that will be used as parameters for the various nodes in our graph. This includes a filter object that we will pass to our filtering node, and an Classifier object that we will pass to our classification node.

In [ ]:
# create filter and classifier objects
# these objects will be parameters to the filter and classifier nodes in the graph

# 4th order Butterworth filter with passband of 0.1-8 H
mp_filt = mp.Filter.create_butter(s, 4, (0.1, 8), 'bandpass', 'sos', Fs)

# LDA classifier
mp_clsf = mp.Classifier.create_LDA(s, shrinkage='auto', solver='lsqr')

Now, we will create our graph and add our various nodes to the graph using the ```add_to_graph()``` factory method. 

For our fNirs pipeline, we will use nodes to filter our data, epoch the data, compute and concatenate features, flatten/normalize/select features, and classify.

In [ ]:
# create the graph and add nodes
g = mp.Graph.create(s)

# filter the data
mp.kernels.FiltFiltKernel.add_to_graph(g, t_data_in, mp_filt, v_tensors[0],
                                       init_input=t_tr_data,  # Note inserting the training data here so that it goes through the entire graph
                                       init_labels=t_tr_labels)

# epoch the data
mp.kernels.EpochKernel.add_to_graph(g, v_tensors[0], v_tensors[1],
                                    epoch_len=epoch_len,
                                    epoch_stride=epoch_stride,
                                    axis=1)

# compute features
mp.kernels.MeanKernel.add_to_graph(g, v_tensors[1], v_tensors[2],
                                   axis=2, keepdims=True)

mp.kernels.VarKernel.add_to_graph(g, v_tensors[1], v_tensors[3],
                                  axis=2, keepdims=True)

mp.kernels.KurtosisKernel.add_to_graph(g, v_tensors[1], v_tensors[4],
                                       axis=2, keepdims=True)

mp.kernels.SkewnessKernel.add_to_graph(g, v_tensors[1], v_tensors[5],
                                       axis=2, keepdims=True)

mp.kernels.SlopeKernel.add_to_graph(g, v_tensors[1], v_tensors[6], Fs=Fs)

# concatenation the features
mp.kernels.ConcatenationKernel.add_to_graph(g, v_tensors[2], v_tensors[3], v_tensors[7], axis=2)
mp.kernels.ConcatenationKernel.add_to_graph(g, v_tensors[7], v_tensors[4], v_tensors[8], axis=2)
mp.kernels.ConcatenationKernel.add_to_graph(g, v_tensors[8], v_tensors[5], v_tensors[9], axis=2)
mp.kernels.ConcatenationKernel.add_to_graph(g, v_tensors[9], v_tensors[6], v_tensors[10], axis=2)

# flatten the feature vector
mp.kernels.ReshapeKernel.add_to_graph(g, v_tensors[10], v_tensors[11], (-1,))

# normalize the features
mp.kernels.FeatureNormalizationKernel.add_to_graph(g, v_tensors[11], v_tensors[12], axis=0)

# select features
mp.kernels.FeatureSelectionKernel.add_to_graph(g, v_tensors[12], v_tensors[13], k=100)

# classify
mp.kernels.ClassifierKernel.add_to_graph(g, v_tensors[13], mp_clsf, t_pred)

From the input XDF source we will extract the test labels into a tensor by using the ```load_into_tensors()``` method. This method returns 2-4 tensors with the second one being the marker data, so we will use the index 1 to access these labels.

In [ ]:
# save the testing labels for later
t_te_labels = te_data_src.load_into_tensors()[1]

We will then verify the graph using the ```verify()``` method. Verifying the graph orders the nodes for execution and ensure that the inputs and outputs of each processing node are appropriately typed and sized.

In [ ]:
# verify the graph
g.verify()

Using the prediction outputed from our classifier node, we will perform cross validation on our graph and print the accuracy.

The next step is to initialize the graph using the ```initialize()``` method. This step is required for pipelines that have methods that need to be trained or fit.

In [ ]:
# cross validate and init graph
cv_acc = g.cross_validate(t_pred)
print(f"Cross validation accuracy {cv_acc:0.3f}")

g.initialize()

We are now ready to run our pipeline. To run the graph for the provided input data, we use the ```execute()``` method. Since we are using epoched/class-seperated data, the trial labels are known, so we can pass them into the execute method. We will run the graph twice, once for each test label, and print the true and predicted label.

In [ ]:
# run the graph twice, once for each test trial
for i_t, label in enumerate(t_te_labels.data):
    g.execute(label=label)
    print(f"Trial {i_t+1} - True label: {label}, predicted label: {t_pred.data[0]}")